# Implementation of Finite Elements

This tutorial shows how to implement our own finite elements in C++,
and how to use them within the NGSolve language. We implement first order and second order triangular finite elements.

* Finite elements implement the basis functions:
[myElement.hpp](http://localhost:8888/edit/myElement.hpp) 
[myElement.cpp](http://localhost:8888/edit/myElement.cpp)

* Differential operators define the mapping from coefficients to function values:
[myDiffOp.hpp](http://localhost:8888/edit/myDiffOp.hpp) 

* Finite element spaces implement the enumeration of degrees of freedom, and creation of elements:
[myFESpace.hpp](http://localhost:8888/edit/myFESpace.hpp) 
[myFESpace.cpp](http://localhost:8888/edit/myFESpace.cpp)

We combine all cpp files to one string

In [ ]:
from pathlib import Path
txt = Path('myElement.cpp').read_text() + \
    Path('myFESpace.cpp').read_text() + \
    Path('mymodule.cpp').read_text() 
# print (txt)

and let NGSolve call the compiler, and load the new library as a Python module:

In [ ]:
from ngsolve.fem import CompilePythonModule
m = CompilePythonModule(txt, init_function_name='mymodule', add_header=False)

In [ ]:
from netgen.occ import unit_square
from ngsolve import *
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2))

We can now create an instance of our own finite element space

In [ ]:
fes = m.MyFESpace(mesh, secondorder=True, dirichlet=".*")

and use it within NGSolve such as the builtin finite element spaces:

In [ ]:
print ("ndof = ", fes.ndof)

In [ ]:
gfu = GridFunction(fes)
gfu.Set(x*y)
from ngsolve.webgui import Draw
Draw (gfu)
Draw (grad(gfu)[0], mesh);

and solve the standard problem:

In [ ]:
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx).Assemble()
f = LinearForm(1*v*dx).Assemble()
gfu.vec.data = a.mat.Inverse(fes.FreeDofs())*f.vec
Draw (gfu);